# SQL Mock Client Project

### Introduction

Writing a SQL query for a client request using mock data.

I will be using Pandas to show the tables and output, which are saved as csv files in the data folder.

In [8]:
import pandas as pd

### Client's request

The client wants data on how salaries have changed at specific companies over time. They specifically want monthly timeseries data that shows the average salary for each company at the MSA level. The companies they are interested in are in the **client_company_list** table.

### Data

Using [Mockaroo](https://www.mockaroo.com/), I created mock data for each table.
* **positions** table: 1,000 rows
* **predicted_salaries** table: 420 rows
* **company_refs** table: 4 rows
* **client_company_list** table: 3 rows

**positions** table

In [14]:
df = pd.read_csv('data/positions.csv')
df.head(10)

,user_id,position_id,company_id,title,mapped_role,msa,startdate,enddate
0,1,1,3,Desktop Support Technician,Cashier,Seattle WA,2019-05-12 06:15:30,NaN
1,2,2,2,Business Systems Development Analyst,Lawyer,San Francisco CA,2018-02-16 16:52:42,NaN
2,3,3,3,Marketing Manager,Lawyer,San Francisco CA,2019-01-02 00:23:45,2022-03-15 23:57:29
3,4,4,4,VP Marketing,Lawyer,San Francisco CA,2020-02-11 21:18:29,2021-09-08 19:46:29
4,5,5,3,Social Worker,Truck Driver,Seattle WA,2018-04-06 20:29:13,2021-06-02 05:17:33
5,6,6,4,Quality Control Specialist,Lawyer,Seattle WA,2020-08-16 14:34:26,2022-08-02 01:27:00
6,7,7,3,Systems Administrator III,Software Engineer,San Francisco CA,2018-07-06 10:38:38,2022-03-31 08:50:40
7,8,8,1,Media Manager I,Data Scientist,Seattle WA,2018-10-26 18:29:20,2022-02-28 21:22:24
8,9,9,3,Statistician III,Cashier,Seattle WA,2020-05-23 20:02:30,2022-02-09 03:12:48
9,10,10,3,Product Engineer,Truck Driver,Seattle WA,2018-04-16 02:23:59,2022-02-18 22:23:16


**predicted_salaries** table

In [13]:
df = pd.read_csv('data/predicted_salaries.csv')
df.head(10)

,company_id,mapped_role,msa,year,salary
0,1,Software Engineer,New York NY,2018,110000.0
1,1,Salesperson,New York NY,2018,70000.0
2,1,Data Scientist,New York NY,2018,100000.0
3,1,Consultant,New York NY,2018,80000.0
4,1,Lawyer,New York NY,2018,200000.0
5,1,Truck Driver,New York NY,2018,70000.0
6,1,Cashier,New York NY,2018,50000.0
7,1,Software Engineer,New York NY,2019,112200.0
8,1,Salesperson,New York NY,2019,71400.0
9,1,Data Scientist,New York NY,2019,102000.0


**company_refs** table

In [11]:
df = pd.read_csv('data/company_refs.csv')
df.head()

,company_id,name
0,1,Apple
1,2,Meta
2,3,Netflix
3,4,Amazon


**client_company_list** table

In [12]:
df = pd.read_csv('data/client_company_list.csv')
df.head()

,company_name
0,Apple
1,Meta
2,Amazon


### Creating a database in PostgreSQL

I made a new database in PostgreSQL, then used SQL Shell to create the four tables and insert the data.

![tables](images/tables.png)

### SQL Code (based on PostgreSQL 15)
~~~sql

-- Creating a timeseries CTE with intervals of 1 month 
WITH timeseries AS (
    SELECT * 
    FROM generate_series(
        (SELECT MIN(startdate) FROM positions), -- Minimum value
        (SELECT CASE WHEN 
            SUM(CASE WHEN enddate IS NULL THEN 1 ELSE 0 END) > 0 THEN CURRENT_DATE 
            ELSE MAX(enddate) END 
        FROM positions), -- Maximum value: if there is a NULL in enddate then the maximum value is the current date
        INTERVAL '1 month'
    ) AS year_month
),

-- Creating a CTE to get IDs of the companies that the client has requested
company_list AS (
    SELECT company_id, name 
    FROM company_refs 
    WHERE LOWER(name) IN (SELECT LOWER(company_name) FROM client_company_list)
),

-- Using previous CTEs, creating a CTE to count the number of people in each role based on company, location, and month, and cross joining with the timeseries CTE
role_temp AS (
    SELECT p.company_id, 
        p.msa, 
        year_month, 
        p.mapped_role, 
        COUNT(p.mapped_role) AS role_count -- Counting the number of people in each role based on company, location, and month
    FROM company_list cl 
    LEFT JOIN positions p ON cl.company_id = p.company_id -- Left joining to only include requested companies
    CROSS JOIN timeseries -- Cross joining with the timeseries to multiply the rows for each month
    WHERE year_month BETWEEN startdate AND (CASE WHEN enddate IS NULL THEN CURRENT_DATE ELSE enddate END) 
    GROUP BY p.company_id, p.msa, year_month, mapped_role
),

-- Using the previous CTE, creating a CTE to calculate weighted salaries and counts for each role
salary_temp AS (
    SELECT t.company_id, 
        t.msa, 
        t.year_month, 
        t.mapped_role, 
        ps.salary * t.role_count AS total_salary, -- Will be used to calculate numerator value
        t.role_count AS total_count -- Will be used to calculate denomintor value
    FROM role_temp t 
    LEFT JOIN predicted_salaries ps ON t.company_id = ps.company_id
        AND t.msa = ps.msa 
        AND t.mapped_role = ps.mapped_role 
        AND EXTRACT(year FROM t.year_month) = ps.year
),

-- Using the previous CTE, creating a CTE to sum the salary and count values based on company, msa, and month
monthly_sums_temp AS (
    SELECT company_id, 
        msa, 
        year_month, 
        SUM(total_salary) AS salary_sum, -- Calculating the numerator value
        SUM(total_count) AS count_sum -- Calculating the denominator value
    FROM salary_temp 
    GROUP BY company_id, msa, year_month
)

SELECT mst.company_id, 
    name AS company_name, -- Adding the company name for client's ease of use 
    msa, 
    TO_CHAR(year_month, 'yyyy-mm') AS year_month, -- Formatting to remove days
    ROUND(salary_sum / count_sum, 2) AS avg_salary -- Calculating the average salary for each company, msa, and month
FROM monthly_sums_temp mst 
LEFT JOIN company_list cl ON mst.company_id = cl.company_id -- Joining with previous company CTE to add the company names
ORDER BY year_month ASC, company_id ASC; -- Ordering by year and month first, then company ID
~~~



#### Output

The following is the output of the SQL code saved in csv format.

In [16]:
df = pd.read_csv('data/sql_output.csv')
df.head(20)

,company_id,company_name,msa,year_month,avg_salary
0,2,Meta,San Francisco CA,2018-01,110000.00
1,1,Apple,New York NY,2018-02,85000.00
2,1,Apple,San Francisco CA,2018-02,80000.00
3,1,Apple,Seattle WA,2018-02,83333.33
4,2,Meta,New York NY,2018-02,50000.00
5,2,Meta,San Francisco CA,2018-02,82500.00
6,2,Meta,Seattle WA,2018-02,72000.00
7,4,Amazon,Seattle WA,2018-02,103333.33
8,4,Amazon,New York NY,2018-02,180000.00
9,1,Apple,San Francisco CA,2018-03,102857.14
